<a href="https://colab.research.google.com/github/joblazek/psp-auction/blob/main/psp_continuous.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
# Networked Implementation of Algorithm 1

import threading
import time
import math
import numpy as np
from typing import Dict, Tuple, List
import pandas as pd
from IPython.display import display, Markdown


In [42]:
class Seller:
    """A virtual seller to hold bids and compute allocations."""
    def __init__(self,
                 Q_max: float):
        """
        :param Q_max: total supply at this node
        :param s = {i: s_i}: Bid profile: (q_i, p_i) for buyer i
        """
        self.Q_max = Q_max
        self.s: Dict[str, Tuple[float,float]] = {}
        self.lock = threading.Lock()
        self.listeners: List['buyer'] = []

    def update_bid(self,
                   i: str,
                   s_i: Tuple[float, float]):
        """Update the bid profile s_i = (q_i, p_i) for buyer i; notify all listeners."""
        with self.lock:
            self.s[i] = s_i
            # Notify buyers so they can recompute
            for listener in self.listeners:
                listener.notify_update()

    def register_listener(self,
                          buyer: 'buyer'):
        """buyers call this to receive bid-update notifications."""
        self.listeners.append(buyer)

    def get_s_hat_minus(self,
                        i: str) -> Dict[str, Tuple[float, float]]:
        """Retrieve ŝ_{-i}: bids of all buyers except i."""
        with self.lock:
            return {j: s for j, s in self.s.items() if j != i}

    def allocate_and_price(self) -> Tuple[str, float]:
        """
        Compute allocation a_i and payment c_i for a single-unit second-price auction.
        - Winner: a_i = argmax_j p_j
        - Payment: second-highest price
        Returns (winner_id, payment).
        """
        with self.lock:
            if not self.s:
                return None, 0.0
            sorted_by_price = sorted(self.s.items(), key=lambda x: x[1][1], reverse=True)
            winner, _ = sorted_by_price[0]
            payment = sorted_by_price[1][1] if len(sorted_by_price) > 1 else 0.0
            return winner, payment


In [43]:
# Buyer implements Algorithm 1,
class Buyer:
    """
    Implements Algorithm 1 (Lazar & Semret, Appendix B),
    listens for seller updates.

    Initialization:
        s_i = 0
        ŝ_{-i} = ∅

    1. Compute truthful ε-best reply t_i = (v_i, w_i):

        v_i = [ sup G_i(ŝ_{-i}) – ε / θ_i′(0) ]_+

        w_i = θ_i′(v_i)

        where sup G_i(ŝ_{-i}) is

        sup { z ∈ [0, Q] :

              z ≤ Q_i(θ_i′(z), ŝ_{-i})

           and ∫₀ᶻ P_i(ζ, ŝ_{-i}) dζ ≤ b_i }

    2. If u_i(t_i, ŝ_{-i}) > u_i(s_i, ŝ_{-i}) + ε, then

           s_i ← t_i

    3. Sleep 1 second and repeat.
    """
class Buyer:
    def __init__(self,
                 i: str,
                 epsilon: float,
                 b_i: float,
                 q_i: float,
                 kappa_i: float,
                 seller: Seller):
        """
        :param i: buyer id
        :param epsilon: threshold ε
        :param b_i: budget
        :param q_i: max physical quantity
        :param kappa_i: valuation intensity
        :param seller: the seller this buyer bids on
        """
        self.i = i
        self.epsilon = epsilon
        self.b_i = b_i
        self.q_i = q_i
        self.kappa_i = kappa_i
        self.seller = seller

        # Current bid (q_i, p_i)
        self.s_i: Tuple[float,float] = (0.0, 0.0)
        self.seller.update_bid(self.i, self.s_i)

        # Concurrency primitives
        self.update_event = threading.Event()
        self.running = True

        # Register for updates
        self.seller.register_listener(self)

    def notify_update(self):
        """Called by seller when any bid updates."""
        self.update_event.set()

    def theta_i(self,
                z: float
                ) -> float:
        """
        Valuation function θ_i(z).
        We are using the parabolic valuation function:

            θ_i(z) = (κ_i / 2) * (min(z, q_i))^2
                     + κ_i * q_i * (min(z, q_i))

        The valuation function is strictly concave for z≤qi​,
        ensuring diminishing marginal returns.
        Beyond qi​, the valuation function becomes flat,
        reflecting the physical limit of the resource.

        :param z: allocated quantity
        :returns: utility θ_i(z)
        """
        m = min(z, self.q_i)
        return 0.5 * self.kappa_i * m**2 + self.kappa_i * self.q_i * m

    def theta_i_prime(self,
                      z: float) -> float:
        """
        Marginal valuation θ_i′(z).
        i.e., the derivative of the parabolic valuation function θ_i:

            θ_i(z) = (κ_i / 2) * (min(z, q_i))^2
                     + κ_i * q_i * min(z, q_i)
        ⇒
            θ_i'(z)
            = κ_i * (z + q_i),    for 0 ≤ z < q_i
            = 0,                  for z ≥ q_i
        """
        if z < self.q_i:
            return self.kappa_i * (z + self.q_i)
        else:
            return 0.0

    def Q_i(self,
            p_i: float,
            s_hat_minus: Dict[str, Tuple[float, float]]) -> float:
        """
        Conditional raw supply curve Q̄_i(p_i; s_{-i}):

        Given your bid price p_i and the profile of opponents' bids s_{-i},
        this returns the maximum quantity available to you after fully
        serving all opponents whose bids exceed p_i.

            Q̄_i(p_i; s_{-i}) = max { Q_max - ∑_{j: p_j >= p_i} q_j, 0 }

        where each opponent j requests quantity q_j at price p_j.
        """
        # Start with full capacity
        remaining = self.seller.Q_max
        # Subtract the quantities of all opponents
        # whose bid price p_j strictly exceeds p_i
        for (q_j, p_j) in s_hat_minus.values():
            if p_j >= p_i:
                remaining -= q_j
        # Cannot go below zero
        return max(remaining, 0.0)

    def sup_G_discrete(self,
                       s_hat_minus: Dict[str, Tuple[float, float]]
                      ) -> float:
        """
        Discrete sup G_i(s_{-i}):

            G_i = {z : z ≤ Q_i(θ'(z),s_hat)  and  cost_at_z(z) ≤ b_i}

        We only check z at the "breakpoints" from the price ladder:
            z_k = ∑_{ℓ=1}^k slice_ℓ

        Returns sup G_i among these discrete points.
        """
        best_z = 0.0
        # Build the price ladder slices
        ladder = self.build_price_ladder(s_hat_minus)
        # Generate candidate z's at cumulative slice boundaries
        candidates = [0.0]
        cum = 0.0
        for slice_size, _ in ladder:
            cum += slice_size
            candidates.append(cum)

        # Evaluate feasibility at each candidate z
        for z in candidates:
            price = self.theta_i_prime(z)
            if z <= self.Q_i(price, s_hat_minus) and self.cost_at_z(z, s_hat_minus) <= self.b_i:
                best_z = z
        return best_z

    def sup_G(self,
              s_hat_minus: Dict[str, Tuple[float, float]]) -> float:
        """
        Compute sup G_i(s_{-i}), where

            G_i(s_{-i}) = { z ∈ [0, Q_max] : z ≤ Q_i(θ'(z); s_{-i})

              and ∫₀ᶻ P_i(ζ; s_{-i}) dζ ≤ b_i }

        Notice that here we call Q_i at the price θ'(z):

            z ≤ Q_i(θ'(z); s_{-i}).
        """
        best_z = 0.0
        for z in np.linspace(0, self.q_i, 100):
            if (z <= self.Q_i(self.theta_i_prime(z), s_hat_minus)
            and self.integral_P(z, s_hat_minus) <= self.b_i):
                best_z = z
        return best_z

    def compute_t_i(self,
                    s_hat_minus: Dict[str, Tuple[float, float]]
                    ) -> Tuple[float, float]:
        """
        Compute t_i = (v_i, w_i) per Proposition 1:

            v_i = [sup_G - ε/θ_i′(0)]_+

            w_i = θ_i′(v_i)
        """
        G_sup = self.sup_G(s_hat_minus)
        adjustment = self.epsilon / self.theta_i_prime(0.0)
        v_i = max(G_sup - adjustment, 0.0)
        w_i = self.theta_i_prime(v_i)
        return v_i, w_i

    def a_i(self,
            s_i: Tuple[float, float],
            s_hat_minus: Dict[str, Tuple[float, float]]) -> float:
        """
        Allocation rule:
            a_i(s) = q_i ∧ Q_i(p_i, ŝ_{-i})
        """
        q_i, p_i = s_i
        return min(q_i, self.Q_i(p_i, s_hat_minus))

    def build_price_ladder(self,
                           s_hat: Dict[str, Tuple[float,float]]
                           ) -> List[Tuple[float,float]]:
        """
        Returns a list of (slice_size, price) in descending-order of price:
          • Opponents with highest bids get served first at their price.
          • Any leftover capacity goes at price 0.
        """
        # 1) Gather opponent bids
        opp = [(qj, pj) for (qj, pj) in s_hat.values()]
        # 2) Sort by price descending
        opp.sort(key=lambda x: x[1], reverse=True)

        ladder = []
        remaining = self.seller.Q_max
        # 3) Carve out each opponent’s slice
        for qj, pj in opp:
            if remaining <= 0:
                break
            serve = min(qj, remaining)
            ladder.append((serve, pj))
            remaining -= serve
        # 4) Remainder at price zero
        if remaining > 0:
            ladder.append((remaining, 0.0))
        return ladder

    def P_i(self,
            z: float,
            s_hat_minus: Dict[str, Tuple[float,float]]) -> float:
        """
        Price density function:

            P_i(z; ŝ_{-i}) = inf { y ≥ 0 : Q_i(y; ŝ_{-i}) ≥ z }.

        We sample candidate prices (0 and all opponents' p_j),
        and return the smallest y meeting Q_i(y) ≥ z,
        but realized as a staircase, i.e. for a given z,
        which price slice are we in?
        """
        cumulative = 0.0
        for qj, pj in sorted(s_hat_minus.values(), key=lambda x: x[1], reverse=True):
            cumulative += qj
            if z <= cumulative:
                return pj
        # if z exceeds all opponent‐slices, price = 0
        return 0.0

    def integral_P(self,
                   z: float,
                   s_hat_minus: Dict[str, Tuple[float, float]]) -> float:
        """Compute

        ∫₀ᶻ P_i(ζ, ŝ_{-i}) dζ via simple trapezoidal rule.

                where:
          - z = allocation to i under PSP (infinitely divisible),

          - P_i(z; ŝ_{-i}) = inf{y ≥ 0 : Q_i(y; ŝ_{-i}) ≥ z}.
        """
        N = 100  # increase resolution for accuracy
        zs = np.linspace(0, z, N + 1)
        Ps = [self.P_i(z, s_hat_minus) for z in zs]
        # trapezoidal rule: sum((P[k] + P[k+1]) / 2 * dz)
        dz = z / N
        total = sum((Ps[k] + Ps[k+1]) * 0.5 for k in range(N)) * dz
        return total

    def P_i(self,
            z: float,
            s_hat_minus: Dict[str, Tuple[float, float]]) -> float:
        """
        Price density function:

            P_i(z; ŝ_{-i}) = inf { y ≥ 0 : Q_i(y; ŝ_{-i}) ≥ z }.

        We sample candidate prices (0 and all opponents' p_j),
        and return the smallest y meeting Q_i(y) ≥ z.
        """
        # 1) Build the set of candidate prices {0} ∪ {p_j for each opponent}
        candidates = sorted({0.0} | {p for (_, p) in s_hat_minus.values()})
        # 2) Scan in increasing order and pick the first y such that Q_i(y) ≥ z
        for y in candidates:
            if self.Q_i(y, s_hat_minus) >= z:
                return y
        return float('inf')

    def c_i(self,
            s_i: Tuple[float,float],
            s_hat_minus: Dict[str, Tuple[float,float]]) -> float:
        """
        Progressive PSP cost exactly as in the math:

            c_i(s) = ∫₀^{a_i(s)} P_i(z) dz = ∑ slice_size_k × price_k

                     (up to your allocation a_i).
        """
        # how much you get
        a = self.a_i(s_i, s_hat_minus)
        return cost(z, s_hat_minus)

    def cost(self,
            z: float,
            s_hat_minus: Dict[str, Tuple[float, float]]) -> float:
        """
        Partial cost up to a continuous allocation z:

            cost_at_z(z) = ∑_{k: cumulative ≤ z} slice_k * price_k
                         + (z - consumed_before_last) * price_last
        """
        ladder = self.build_price_ladder(s_hat_minus)
        cost = 0.0
        consumed = 0.0
        for slice_size, price in ladder:
            if consumed >= z:
                break
            take = min(slice_size, z - consumed)
            cost += take * price
            consumed += take
        return cost

    def c_i_old(self,
            s_i: Tuple[float, float],
            s_hat_minus: Dict[str, Tuple[float, float]]) -> float:
        """
        Cost under progressive allocation:

            c_i(s) = ∫₀^{a_i(s)} P_i(z; ŝ_{-i}) dz

        where:
          - a_i(s) = allocation to i under PSP (infinitely divisible),
          - P_i(z; ŝ_{-i}) = inf{y ≥ 0 : Q_i(y; ŝ_{-i}) ≥ z}.

        We compute this via a trapezoidal rule over N steps.
        """
        # Determine allocation
        a = self.a_i(s_i, s_hat_minus)
        # Numerically integrate P_i from 0 to a
        return self.integral_P(a, s_hat_minus)

    def u_i(self,
            s_i: Tuple[float, float],
            s_hat_minus: Dict[str, Tuple[float, float]]) -> float:
        """
        Utility:
            u_i(s) = θ_i(a_i(s)) − c_i(s)
        """
        a = self.a_i(s_i, s_hat_minus)
        return self.theta_i(a) - self.c_i(s_i, s_hat_minus)

    def step(self) -> bool:
        """
        Perform one Algorithm 1 iteration.
        Returns True if the buyer updated its bid (s_i changed), False otherwise.
        """
        s_hat_minus = self.seller.get_s_hat_minus(self.i)
        t_i = self.compute_t_i(s_hat_minus)
        old = self.s_i
        # Check utility improvement condition
        if self.u_i(t_i, s_hat_minus) > self.u_i(old, s_hat_minus) + self.epsilon:
            self.s_i = t_i
            self.seller.update_bid(self.i, t_i)
            return True
        return False

    def run(self,
            interval: float = 1.0):
        """
        Buyer thread: waits for updates or interval timeout, then steps.
        """
        while self.running:
            # Wait until either notified or interval elapsed
            self.update_event.wait(timeout=interval)
            self.update_event.clear()
            self.step()

    def stop(self):
        self.running = False
        self.update_event.set()


In [44]:
from IPython.display import HTML, display

def print_round_info_multi(round_num: int,
                           sellers: List[Seller],
                           buyers_by_seller: Dict[Seller, List[Buyer]]) -> None:
    """
    Displays auction summary tables for each seller side-by-side.

    :param round_num: current round number
    :param seller: list of auction instances
    :param buyers_by_seller: dict mapping each seller to its list of buyers
    """
    tables_html = []
    for idx, seller in enumerate(sellers, start=1):
        records = []
        buyers = buyers_by_seller[seller]
        for ag in buyers:
            s_hat = seller.get_s_hat_minus(ag.i)
            q, p = ag.s_i
            records.append({
                "buyer":   ag.i,
                "q_i":     f"{q:.2f}",
                "p_i":     f"{p:.2f}",
                "alloc":   f"{ag.a_i(ag.s_i, s_hat):.2f}",
                "cost":    f"{ag.c_i(ag.s_i, s_hat):.2f}",
                "utility": f"{ag.u_i(ag.s_i, s_hat):.2f}"
            })
        winner, raw_pay = seller.allocate_and_price()
        pay_price = raw_pay[1] if isinstance(raw_pay, tuple) else float(raw_pay)
        records.append({
            "buyer":   f"Winner: {winner}",
            "q_i":     "",
            "p_i":     "",
            "alloc":   "",
            "cost":    "",
            "utility": f"Pays {pay_price:.2f}"
        })
        df = pd.DataFrame(records)
        html = df.to_html(index=False, escape=False)
        # Wrap with a div including seller label
        html_wrapped = f"""
            <div style="flex: 1; margin-right: 20px;">
                <h4>Auction on seller {idx}</h4>
                {html}
            </div>
        """
        tables_html.append(html_wrapped)
    # Combine all tables in a flex container
    combined = f"""
        <div style="display: flex; flex-wrap: wrap; align-items: flex-start;">
            {''.join(tables_html)}
        </div>
    """
    display(HTML(f"<h3>Round {round_num} Summary</h3>{combined}"))

In [45]:
def simulation(sellers: List[seller1],
                            buyers_by_seller: Dict[Seller, List[Buyer]],
                            rounds: int = 10,
                            interval: float = 1.0):
    # 1) collect all buyers
    all_buyers = [i for lst in buyers_by_seller.values() for i in lst]

    # 2) start each buyer in its own thread
    threads = []
    for i in all_buyers:
        t = threading.Thread(target=i.run, daemon=True)
        threads.append(t)
        t.start()

    # 3) main reporting loop
    for round_num in range(1, rounds + 1):

        for i in all_buyers:
            i.updated = False

        time.sleep(interval)

        # snapshot current state side-by-side
        print_round_info_multi(round_num, sellers, buyers_by_seller)

        # check global convergence: no buyer updated
        if not any(i.updated for i in all_buyers):
            print(f"**Global convergence after round {round_num}.**")
            break

    # 4) shut down all buyers
    for i in all_buyers:
        i.stop()
    for t in threads:
        t.join()

In [48]:

# Example usage
if __name__ == "__main__":

    # 1) Create sellers
    seller1 = Seller(Q_max=5.0)
    seller2 = Seller(Q_max=8.0)
    seller3 = Seller(Q_max=20.0)

    # 2) Create buyers and assign them
    """
    Buyer params:
        i: buyer id
        epsilon: threshold ε
        b_i: budget
        q_i: max physical quantity
        kappa_i: valuation intensity
        seller: the seller this buyer bids on
    """
    buyers_by_seller = {
        seller1: [
            Buyer("1", 0.1, 10, 1.0, 1.0, seller1),
            Buyer("2", 0.1, 12, 4.5, 1.8, seller1),
        ],
        seller2: [
            Buyer("3", 0.1, 15, 5.0, 1.5, seller2),
            Buyer("4", 0.1, 20, 1.5, 1.2, seller2),
            Buyer("5", 0.1, 16, 1.8, 1.5, seller2),
        ],
        seller3: [
            Buyer("6", 0.1, 8, 2.5, 1.2, seller3),
            Buyer("7", 0.1, 8, 12.5, 1.2, seller3),
        ]
    }

    # 3) Run the multi‐seller simulation with side-by-side tables
    simulation(
        sellers=list(buyers_by_seller.keys()),
        buyers_by_seller=buyers_by_seller,
        rounds=10,
        interval=1.0
    )


buyer,q_i,p_i,alloc,cost,utility
1,0.41,1.41,0.41,6.55,-6.07
2,4.49,16.18,4.49,0.57,53.91
Winner: 2,,,,,Pays 1.41
buyer,q_i,p_i,alloc,cost,utility
3,4.99,14.98,4.99,0.00,56.05
4,0.00,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00
Winner: 3,,,,,Pays 0.00
buyer,q_i,p_i,alloc,cost,utility
6,0.22,3.26,0.22,6.57,-5.89


**Global convergence after round 1.**
